In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import gensim.downloader as gensim_downloader
import gensim
import multiprocessing
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from joblib import Parallel, delayed
from Source.preprocess_data import *  ## import all functions from preprocess_data.py
from Source.postprocess_data import * ## import all functions from postprocess_data.py
from Source.utils import *  ## import all functions from utils.py
import nltk
import optuna

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix 
from nltk.corpus import stopwords  
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer, SnowballStemmer
from tqdm import tqdm
tqdm.pandas()
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

nw = multiprocessing.cpu_count()



client = MlflowClient(tracking_uri="http://localhost:8080")
os.environ["TF_KERAS"]='1'
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
print("Version TF :", tf.__version__)

c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


2.10.1
Num GPUs Available:  1
GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Version TF : 2.10.1


c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import mlflow
import os

# Choisir le dossier local où enregistrer le modèle
export_path = os.path.join("exp_models", "log_regression_model_opt")
os.makedirs(export_path, exist_ok=True)

# URI du modèle dans le registry, version 1
model_uri = "models:/log_regression_model_opt/1"

# Charger et sauvegarder le modèle localement
mlflow.sklearn.save_model(mlflow.sklearn.load_model(model_uri),
    path=export_path
)

print(f"Modèle exporté dans : {export_path}")

Modèle exporté dans : exp_models\log_regression_model_opt


In [5]:
import mlflow.tensorflow
import os

model_uri = "models:/Best_LSTM_model/1"
export_path = os.path.join("exp_models", "Best_LSTM_model")
os.makedirs(export_path, exist_ok=True)

# Charger le modèle depuis le registry
model = mlflow.tensorflow.load_model(model_uri)

# Sauvegarder localement au format TensorFlow SavedModel
import tensorflow as tf
tf.keras.models.save_model(model, export_path)

print(f"Modèle exporté dans : {export_path}")

INFO:tensorflow:Assets written to: exp_models\Best_LSTM_model\assets


INFO:tensorflow:Assets written to: exp_models\Best_LSTM_model\assets


Modèle exporté dans : exp_models\Best_LSTM_model


In [6]:
import mlflow.tensorflow
import os

model_uri = "models:/Bert_best_Model/1"
export_path = os.path.join("exp_models", "Best_BERT_model")
os.makedirs(export_path, exist_ok=True)

# Charger le modèle depuis le registry
model = mlflow.tensorflow.load_model(model_uri)

# Sauvegarder localement au format TensorFlow SavedModel
import tensorflow as tf
tf.keras.models.save_model(model, export_path)

print(f"Modèle exporté dans : {export_path}")

ValueError: The two structures don't have the same nested structure.

First structure: type=tuple str=(({'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, None, None, None, None, None, None, None, None, None, None, None, None, False), {})

Second structure: type=tuple str=((TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask'), None, None, None, None, None, None, None, None, None, None, None, False), {})

More specifically: Substructure "type=dict str={'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}" is a sequence, while substructure "type=TensorSpec str=TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids')" is not
Entire first structure:
(({'attention_mask': ., 'input_ids': .}, ., ., ., ., ., ., 

In [11]:
model_uri = "models:/Bert_best_Model/1"
model = mlflow.tensorflow.load_model(model_uri)

ValueError: The two structures don't have the same nested structure.

First structure: type=tuple str=(({'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, None, None, None, None, None, None, None, None, None, None, None, None, False), {})

Second structure: type=tuple str=((TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask'), None, None, None, None, None, None, None, None, None, None, None, False), {})

More specifically: Substructure "type=dict str={'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}" is a sequence, while substructure "type=TensorSpec str=TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids')" is not
Entire first structure:
(({'attention_mask': ., 'input_ids': .}, ., ., ., ., ., ., 

In [8]:
import mlflow.tensorflow
import os

model_uri = "models:/bilstm_best/1"
export_path = os.path.join("exp_models", "Best_BiLSTM_model")
os.makedirs(export_path, exist_ok=True)

# Charger le modèle depuis le registry
model = mlflow.tensorflow.load_model(model_uri)

# Sauvegarder localement au format TensorFlow SavedModel
import tensorflow as tf
tf.keras.models.save_model(model, export_path)

print(f"Modèle exporté dans : {export_path}")

INFO:tensorflow:Assets written to: exp_models\Best_BiLSTM_model\assets


INFO:tensorflow:Assets written to: exp_models\Best_BiLSTM_model\assets


Modèle exporté dans : exp_models\Best_BiLSTM_model
